# Validate the outputs! Regression!

In [1]:
# !pip install xgboost
# !pip install shap

In [2]:
import pandas as pd
from rdkit import Chem
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from collections import defaultdict
from sklearn.metrics import mean_squared_error
import shap
import multiprocessing as mp
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


## Get Data

In [3]:
dtypes=defaultdict(lambda:"float")
dtypes['cmp1ID'] = 'category'
dtypes['cmp1name'] = 'category'
dtypes['SMILES'] = 'category'
df = pd.read_csv("../../data/IL/model_data_with_descriptors.csv", dtype=dtypes, na_values=["na"])
df = df.rename(columns={'Viscosity[Liquid]/Pa&#8226;s':'Viscosity', 
                   'Electrical_conductivity[Liquid]/S/m':"ElecConductivity", 
                   'Specific_density[Liquid]/kg/m3': 'Density'
                  })
print(df.info())
print(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3315 entries, 0 to 3314
Columns: 5674 entries, Temperature/K to chiralPhMoment
dtypes: category(3), float64(5671)
memory usage: 143.5 MB
None
Index(['Temperature/K', 'Pressure/kPa', 'Density', 'cmp1ID', 'cmp1name',
       'SMILES', 'Viscosity', 'ElecConductivity', 'MW', 'AMW',
       ...
       's1_numAroBonds', 's2_numAroBonds', 's3_numAroBonds', 's4_numAroBonds',
       's34_size', 's34_relSize', 's34_phSize', 's34_phRelSize',
       'chiralMoment', 'chiralPhMoment\r'],
      dtype='object', length=5674)


In [4]:
# Feature clean-up
df.dropna(axis=1, how='any', inplace=True)          # Remove any column that contains missing data
df = df.loc[:, ~df.isin([np.inf, -np.inf]).any()]   # Delete columns that contain at least 1 inf
df = df.loc[:, ~df.isnull().any()]                  # Delete columns that contain at least 1 nan
df = df.loc[:, df.nunique() != 1]                   # Delete columns that have all elements the same
columns_with_na = df.columns[df.applymap(lambda x: 'na' in str(x)).any()] # Identify columns with 'na' values in any of their rows
df = df.drop(columns=columns_with_na) # Drop columns with 'na' values

In [5]:
replaces= {}
for col in df.columns:
    if '[' in col or ']' in col or '<' in col:
        replaces[col] = col.replace('[','_').replace(']','_').replace('<','_')
df = df.rename(columns=replaces)        

In [6]:
outputs = ['Viscosity', 'ElecConductivity', 'Density']
irrelevant = ['cmp1ID', 'cmp1name', 'SMILES']
inputs = list(set(df.columns).difference(set(outputs)).difference(set(irrelevant)))

In [7]:
X = df[inputs]
y = df[outputs]

In [8]:
# y['Viscosity'] = np.log(y['Viscosity'])
y['ElecConductivity'] = -np.log(y['ElecConductivity'])


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
# plt.scatter(X['Temperature/K'], y['Viscosity'])
# plt.scatter(X['Temperature/K'], y['ElecConductivity'])
# plt.scatter(X['Temperature/K'], y['Density'])

In [17]:
x_scaler = StandardScaler() # Scale the feature values
X_scaled = pd.DataFrame(x_scaler.fit_transform(X),columns=inputs) # Scale the feature values

In [20]:
y_scaler = StandardScaler() # Scale the feature values
y_scaled = pd.DataFrame(y_scaler.fit_transform(y),columns=outputs) # Scale the feature values

In [23]:
n = 5
selected = set()
for Y_name in outputs:
    params = {
        "tree_method": "gpu_hist",
        "gpu_id":0,
        "objective": "reg:squarederror",
        "n_jobs":mp.cpu_count() // 2,
        "n_estimators":1000
    }
    
    Y = y_scaled[Y_name]
    rf = xgb.XGBRegressor(**params)
    rf.fit(X_scaled, Y)
    rf_importance = rf.feature_importances_
    feature_imp_rf = pd.Series(rf_importance, index=X.columns).sort_values(ascending=False)
    top_n_features_rf = feature_imp_rf.nlargest(n) # Select the top n feature importances for Random Forest
    print(top_n_features_rf)
    selected.update(top_n_features_rf.index)  
    
#     params = {
#         "tree_method": "gpu_hist",
#         "gpu_id":0,
#         "objective": "reg:squarederror",
#         "n_jobs":mp.cpu_count() // 2,
#         "n_estimators":1000
#     }
    X_restricted = X_scaled[top_n_features_rf.index]
    X_train, X_test, y_train, y_test = train_test_split(X_restricted, Y, random_state=1)
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(f"RMSE of the base model: {mean_squared_error(y_test, preds, squared=False):.3f}")
    print(f"R^2 of the base model: {r2_score(y_test, preds):.3f}")

SM10_EA(ed)       0.242402
SM5_B(p)          0.116643
P_VSA_charge_1    0.041760
CATS2D_03_DA      0.031090
meanDistFromCC    0.028607
dtype: float32
RMSE of the base model: 0.838
R^2 of the base model: 0.056
P_VSA_MR_1      0.138849
CATS2D_03_DA    0.087598
MATS1i          0.070741
H-046           0.061398
AVS_B(m)        0.054600
dtype: float32
RMSE of the base model: 0.501
R^2 of the base model: 0.707
AMW            0.496644
Me             0.112348
Eta_D_epsiA    0.082539
Eta_epsi_A     0.032160
Ui             0.020376
dtype: float32
RMSE of the base model: 0.135
R^2 of the base model: 0.981


In [12]:
print(selected)

{'SpMAD_Dz(p)', 'SpMax4_Bh(p)', 'CATS2D_04_AA', 'CATS2D_01_NN', 'P_VSA_v_2', 'Eta_sh_p', 'P_VSA_LogP_2', 'SM02_EA(ri)', 'minssCH2', 'ATSC8p', 'AVS_B(m)', 'SM11_AEA(ed)', 'SM5_L', 'P_VSA_m_2', 'Eig05_AEA(ed)', 'Cl-102', 'SM12_EA(dm)', 'C-001', 'Mp', 'ChiA_B(m)', 'SM10_EA(ed)', 'minssO', 'SM11_EA(ri)', 'P_VSA_p_2', 'AVS_D/Dt', 'P_VSA_s_6', 'MaxtsC', 'P_VSA_e_3', 'ATS4m', 'SM05_AEA(ed)', 'DECC', 'MaxssO', 'Vx', 'P_VSA_MR_1', 'ATSC2e', 'SM4_B(p)', 'EE_B(p)', 'P_VSA_LogP_6', 'GATS6m', 'nHet', 'TPSA(Tot)', 'EE_Dz(Z)', 'P_VSA_charge_1', 'GATS2m', 'Eig09_AEA(ri)', 'nH', 'J_Dt', 'Chi_Dz(i)', 'SpPosA_B(m)', 'MATS4i', 'CATS2D_03_DA', 'Eig01_EA(bo)', 'IC2', 'PCR', 'GATS4p', 'SHED_PL', 'SM6_X', 'gmax', 'Eta_betaS_A', 'LOC', 'Eta_D_beta_A', 'F05_N-N_', 'MDEC-11', 'CATS2D_00_DD', 'ATSC1e', 'SpMin8_Bh(m)', 'nN', 'Eig12_EA(dm)', 'F04_N-N_', 'Eta_D_epsiA', 'MATS1i', 'Eig06_EA(ed)', 'P_VSA_e_2', 'SM6_B(i)', 'CATS2D_03_LL', 'B09_C-C_', 'MaxsCH3', 'SM2_B(m)', 'Eta_B_A', 'AVS_Dz(Z)', 'Eig11_EA(ed)', 'TPSA(N

In [13]:
X_restricted = X[list(selected)]
X_restricted = scaler.fit_transform(X_restricted) # Scale the feature values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_restricted, y, random_state=1)

In [ ]:
dtrain_reg = xgb.DMatrix(X_train, y_train)
dtest_reg = xgb.DMatrix(X_test, y_test)

In [ ]:
n = 10000
params = {
    "eta": 0.05,
    "max_depth": 10,
    "tree_method": "gpu_hist",
    "gpu_id":0,
    "objective": "reg:squarederror",
    "n_jobs":mp.cpu_count() // 2,
    "n_estimators":n,
#     "device": "cuda",
#     "enable_categorical":True,
#     "max_cat_to_onehot":1, 
}
# params = {
#     "objective": "reg:squarederror", 
#     "tree_method": "gpu_hist", 
#     "gpu_id":0, 
#     "max_cat_to_onehot":1, 
#     "enable_categorical":True,
#     "n_estimators":n,
#     "predictor": "gpu_predictor",
# }
# params = {"objective": "reg:squarederror", "tree_method": "hist"}

model = xgb.XGBRegressor(**params)
model.fit(X_train, y_train)
# model = xgb.train(
#     params=params,
#     dtrain=dtrain_reg,
#     num_boost_round=n,
# )

In [ ]:
preds = model.predict(X_test)
rmse = mean_squared_error(y_test, preds, squared=False)
print(f"RMSE of the base model: {rmse:.3f}")

In [ ]:
print(r2_score(y_test, preds))

In [ ]:
# shap_values = model.predict(X_test, pred_contribs=True)

In [ ]:
explainer = shap.Explainer(model.predict, X_test, max_evals=20000)
shap_values = explainer(X_test)
shap.plots.bar(shap_values)

In [ ]:
# shap_interaction_values = model.predict(dtrain_reg, pred_interactions=True)

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)[0]
print(shap_values)

shap.force_plot(
    explainer.expected_value,
    shap_values[0, :],
    X_train[0, :],
    matplotlib=True,
)